In [9]:
import panel as pn
import pandas as pd
import hvplot.pandas

pn.extension()

df = pd.read_csv('C:\\Becas-Nuevas-Nacionales.csv', encoding='latin1')

df['SEXO'] = df['SEXO'].str.strip().str.capitalize()  # Eliminar espacios y capitalizar
df['NIVEL'] = df['NIVEL'].str.strip()
df['ENTIDAD DESTINO'] = df['ENTIDAD DESTINO'].str.strip()
df['ÁREA DEL CONOCIMIENTO'] = df['ÁREA DEL CONOCIMIENTO'].str.strip()

nivel_select = pn.widgets.Select(name='Nivel', options=['Todos'] + list(df['NIVEL'].unique()), width=200)
entidad_select = pn.widgets.Select(name='Entidad', options=['Todos'] + list(df['ENTIDAD DESTINO'].unique()), width=200)
sexo_select = pn.widgets.Select(name='Sexo', options=['Todos'] + list(df['SEXO'].unique()), width=200)


@pn.depends(nivel=nivel_select, entidad=entidad_select, sexo=sexo_select)
def update_plot(nivel, entidad, sexo):
    filtered_df = df.copy()
    if nivel != 'Todos':
        filtered_df = filtered_df[filtered_df['NIVEL'] == nivel]
    if entidad != 'Todos':
        filtered_df = filtered_df[filtered_df['ENTIDAD DESTINO'] == entidad]
    if sexo != 'Todos':
        filtered_df = filtered_df[filtered_df['SEXO'] == sexo]

    if not filtered_df.empty:
        becas_por_area_filtered = filtered_df['ÁREA DEL CONOCIMIENTO'].value_counts().reset_index()
        becas_por_area_filtered.columns = ['ÁREA DEL CONOCIMIENTO', 'TOTAL DE BECAS']
        plot = becas_por_area_filtered.hvplot.bar(
            x='ÁREA DEL CONOCIMIENTO', 
            y='TOTAL DE BECAS', 
            title=f'Becas por Área del Conocimiento ({nivel} - {entidad} - {sexo})', 
            xlabel='Área del Conocimiento', 
            ylabel='Total de Becas', 
            color='pink',
            width=800, height=400,
            legend=False,  # Sin leyenda
            fontsize={'labels': '10pt'}  # Tamaño de la fuente
        ).opts(xrotation=45)  # Rotación de etiquetas del eje x

        return pn.pane.HoloViews(plot)
    else:
        return pn.pane.HoloViews(hvplot.empty(title='No hay datos disponibles'))
        
dashboard_dinamico = pn.Column(
    'Dashboard Dinámico de Becas por Área del Conocimiento',
    pn.Row(nivel_select, entidad_select, sexo_select, align='center'),
    update_plot,
    align='center',
    width=850, height=600
)
dashboard_dinamico.servable()


Column(align='center', height=600, sizing_mode='fixed', width=850)
    [0] Markdown(str)
    [1] Row(align='center')
        [0] Select(name='Nivel', options=['Todos', 'MAESTRIA', ...], value='Todos', width=200)
        [1] Select(name='Entidad', options=['Todos', 'CIUDAD DE MEXIC...], value='Todos', width=200)
        [2] Select(name='Sexo', options=['Todos', 'Femenino', ...], value='Todos', width=200)
    [2] ParamFunction(function, _pane=HoloViews, defer_load=False)